<a href="https://colab.research.google.com/github/areias/healthcare_fraud/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install wandb -qqq


     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 180 kB 57.7 MB/s 
     |████████████████████████████████| 97 kB 5.2 MB/s 
     |████████████████████████████████| 140 kB 57.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import wandb

In [30]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
train_ben=pd.read_csv('Train_Beneficiarydata-1542865627584.csv')
train_in=pd.read_csv('Train_Inpatientdata-1542865627584.csv')
train_out=pd.read_csv('Train_Outpatientdata-1542865627584.csv')
train=pd.read_csv('Train-1542865627584.csv')
test_ben=pd.read_csv('Test_Beneficiarydata-1542969243754.csv')
test_in=pd.read_csv('Test_Inpatientdata-1542969243754.csv')
test_out=pd.read_csv('Test_Outpatientdata-1542969243754.csv')
test=pd.read_csv('Test-1542969243754.csv')

In [ ]:
test

,Provider
0,PRV51002
1,PRV51006
2,PRV51009
3,PRV51010
4,PRV51018
...,...
1348,PRV57713
1349,PRV57726
1350,PRV57745
1351,PRV57749


In [26]:
train

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No
...,...,...
5405,PRV57759,No
5406,PRV57760,No
5407,PRV57761,No
5408,PRV57762,No


In [ ]:
train

,Provider,PotentialFraud
0,PRV51001,No
1,PRV51003,Yes
2,PRV51004,No
3,PRV51005,Yes
4,PRV51007,No
...,...,...
5405,PRV57759,No
5406,PRV57760,No
5407,PRV57761,No
5408,PRV57762,No


**Merging the data**

**Train**

In [ ]:
train_in['Claim_type']='Inpatient'
train_out['Claim_type']='Outpatient'

In [ ]:
final_train=pd.concat([train_in, train_out])
final_train=pd.merge(train_ben, final_train, on='BeneID')
final_train=pd.merge(final_train, train, on='Provider')

**Test**

In [ ]:
test_in['Claim_type']='Inpatient'
test_out['Claim_type']='Outpatient'

In [ ]:
final_test=pd.concat([test_in, test_out])
final_test=pd.merge(test_ben, final_test, on='BeneID')
final_test=pd.merge(final_test, test, on='Provider')

In [ ]:
final_train.head()

,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,ClmAdmitDiagnosisCode,DeductibleAmtPaid,DischargeDt,DiagnosisGroupCode,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,ClmDiagnosisCode_4,ClmDiagnosisCode_5,ClmDiagnosisCode_6,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,Claim_type,PotentialFraud
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,7866,1068.0,2009-04-18,201,1970,4019,5853,7843,2768,71590,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inpatient,Yes
1,BENE12511,1944-09-01,NaN,1,1,0,39,510,12,12,2,2,2,2,2,2,1,1,1,2,2,0,0,1250,90,CLM724430,2009-12-11,2009-12-11,PRV55912,600,PHY420794,NaN,NaN,NaN,NaN,0.0,NaN,NaN,V560,V4283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient,Yes
2,BENE14577,1935-10-01,NaN,2,1,Y,33,70,12,12,2,1,1,2,2,2,2,1,2,2,1,0,0,2390,440,CLM567180,2009-09-07,2009-09-07,PRV55912,20,PHY381733,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient,Yes
3,BENE15074,1932-06-01,NaN,2,1,0,39,470,12,12,1,2,1,2,2,2,1,1,2,2,2,0,0,870,210,CLM259984,2009-03-22,2009-03-22,PRV55912,30,PHY371570,NaN,PHY393406,NaN,NaN,0.0,NaN,NaN,V812,40390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient,Yes
4,BENE16973,1932-05-01,NaN,1,1,0,39,310,12,12,1,1,1,2,2,2,2,1,2,2,2,24000,2136,450,200,CLM565430,2009-09-06,2009-09-06,PRV55912,50,PHY365867,PHY327147,NaN,NaN,NaN,0.0,NaN,NaN,V7183,53081,78959,4280,E8788,78079,79902,25002,71848,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Outpatient,Yes


In [ ]:
final_train.shape

(558211, 56)

In [ ]:
final_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558211 entries, 0 to 558210
Data columns (total 56 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           558211 non-null  object 
 1   DOB                              558211 non-null  object 
 2   DOD                              4131 non-null    object 
 3   Gender                           558211 non-null  int64  
 4   Race                             558211 non-null  int64  
 5   RenalDiseaseIndicator            558211 non-null  object 
 6   State                            558211 non-null  int64  
 7   County                           558211 non-null  int64  
 8   NoOfMonths_PartACov              558211 non-null  int64  
 9   NoOfMonths_PartBCov              558211 non-null  int64  
 10  ChronicCond_Alzheimer            558211 non-null  int64  
 11  ChronicCond_Heartfailure         558211 non-null  int64  
 12  Ch

In [ ]:
final_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135392 entries, 0 to 135391
Data columns (total 55 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   BeneID                           135392 non-null  object 
 1   DOB                              135392 non-null  object 
 2   DOD                              1040 non-null    object 
 3   Gender                           135392 non-null  int64  
 4   Race                             135392 non-null  int64  
 5   RenalDiseaseIndicator            135392 non-null  object 
 6   State                            135392 non-null  int64  
 7   County                           135392 non-null  int64  
 8   NoOfMonths_PartACov              135392 non-null  int64  
 9   NoOfMonths_PartBCov              135392 non-null  int64  
 10  ChronicCond_Alzheimer            135392 non-null  int64  
 11  ChronicCond_Heartfailure         135392 non-null  int64  
 12  Ch

**Usage of the date variables:**

In [ ]:
# Convert the column Date into Datetime64 datatype
final_train['ClaimStartDt'] = pd.to_datetime(final_train['ClaimStartDt'], format = '%Y-%m-%d')
final_train['ClaimEndDt'] = pd.to_datetime(final_train['ClaimEndDt'], format = '%Y-%m-%d')
final_test['ClaimStartDt'] = pd.to_datetime(final_test['ClaimStartDt'], format = '%Y-%m-%d')
final_test['ClaimEndDt'] = pd.to_datetime(final_test['ClaimEndDt'], format = '%Y-%m-%d')

In [ ]:
#train
final_train['ClaimStartDay']=final_train.ClaimStartDt.dt.day
final_train['ClaimStartMonth']=final_train.ClaimStartDt.dt.month
final_train['ClaimStartYear']=final_train.ClaimStartDt.dt.year
final_train['ClaimEndDay']=final_train.ClaimEndDt.dt.day
final_train['ClaimEndMonth']=final_train.ClaimEndDt.dt.month
final_train['ClaimEndYear']=final_train.ClaimEndDt.dt.year
#test
final_test['ClaimStartDay']=final_test.ClaimStartDt.dt.day
final_test['ClaimStartMonth']=final_test.ClaimStartDt.dt.month
final_test['ClaimStartYear']=final_test.ClaimStartDt.dt.year
final_test['ClaimEndDay']=final_test.ClaimEndDt.dt.day
final_test['ClaimEndMonth']=final_test.ClaimEndDt.dt.month
final_test['ClaimEndYear']=final_test.ClaimEndDt.dt.year


In [ ]:
final_train['ClaimDuration']= final_train['ClaimEndDt']-final_train['ClaimStartDt']
final_test['ClaimDuration']= final_test['ClaimEndDt']-final_test['ClaimStartDt']

0         6 days
1         0 days
2         0 days
3         0 days
4         0 days
           ...  
558206    0 days
558207   20 days
558208    0 days
558209    0 days
558210    7 days
Name: ClaimDuration, Length: 558211, dtype: timedelta64[ns]

In [ ]:
model_train=final_train[['IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 'ClaimDuration', 'Claim_type', 'InscClaimAmtReimbursed', 'PotentialFraud']]
model_test=final_test[['IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 'ClaimDuration', 'Claim_type', 'InscClaimAmtReimbursed']]

In [ ]:
model_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558211 entries, 0 to 558210
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype          
---  ------                    --------------   -----          
 0   IPAnnualReimbursementAmt  558211 non-null  int64          
 1   IPAnnualDeductibleAmt     558211 non-null  int64          
 2   OPAnnualReimbursementAmt  558211 non-null  int64          
 3   OPAnnualDeductibleAmt     558211 non-null  int64          
 4   ClaimDuration             558211 non-null  timedelta64[ns]
 5   Claim_type                558211 non-null  object         
 6   InscClaimAmtReimbursed    558211 non-null  int64          
 7   PotentialFraud            558211 non-null  object         
dtypes: int64(5), object(2), timedelta64[ns](1)
memory usage: 38.3+ MB


In [ ]:
model_train['Claim_type']=model_train['Claim_type'].replace({'Inpatient' : 0, 'Outpatient' : 1 })
model_test['Claim_type']=model_test['Claim_type'].replace({'Inpatient' : 0, 'Outpatient' : 1 })
model_train['PotentialFraud']=pd.get_dummies(model_train['PotentialFraud'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
model_train['Claim_type']=pd.get_dummies(model_train['Claim_type'])
model_test['Claim_type']=pd.get_dummies(model_test['Claim_type'])
model_train['PotentialFraud']=pd.get_dummies(model_train['PotentialFraud'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
model_train['ClaimDuration'] = model_train['ClaimDuration'].dt.days
model_test['ClaimDuration'] = model_test['ClaimDuration'].dt.days

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
target = model_train['PotentialFraud']
model_train = model_train.drop(['PotentialFraud'], axis=1)

In [ ]:
model = LogisticRegression(solver='liblinear', random_state=0)

In [ ]:
model.fit(model_train, target)


LogisticRegression(random_state=0, solver='liblinear')

In [ ]:
x_pred=model.predict(model_train)
y_pred=model.predict(model_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(target, x_pred)

0.6249357321872911

In [31]:
wandb.init(project="visualize-sklearn")

wandb: Currently logged in as: ds_duo (use `wandb login --relogin` to force relogin)


In [32]:
wandb.sklearn.plot_confusion_matrix(target, x_pred)

In [ ]:
wandb.sklearn.plot_classifier(clf, X_train, X_test, y_train, y_test, y_pred, y_probas, labels,
                                                         model_name='SVC', feature_names=None)

In [34]:
wandb.sklearn.plot_learning_curve(model, model_train, target)

In [36]:
y_proba=model.predict_proba(model_train)
y_proba

array([[0.40576962, 0.59423038],
       [0.63311377, 0.36688623],
       [0.63304212, 0.36695788],
       ...,
       [0.41931585, 0.58068415],
       [0.42402954, 0.57597046],
       [0.42727237, 0.57272763]])

In [38]:
wandb.sklearn.plot_roc(target, y_proba, target)

wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.
